In [ ]:
!pip install matplotlib
!pip install nltk

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [4]:
data = pd.read_csv("FinalBalancedDataset.csv")

In [ ]:
data.info()

In [ ]:
data.head(5)

In [7]:
data = data.drop("Unnamed: 0", axis=1)

In [ ]:
data.head(5)

In [ ]:
data['Toxicity'].value_counts()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

## Lemmatizer
1. Leaves
2. Leafs
Leaf

## Text pre-processing

In [11]:
wordnet_lemmatizer = WordNetLemmatizer()

In [12]:
import re

In [13]:
def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = ' '.join(text)
    text = word_tokenize(text)
    text = pos_tag(text)
    lemma = []
    for i in text: lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos = get_wordnet_pos(i[1])))
    lemma = ' '.join(lemma)
    return lemma

In [ ]:
data['clean_tweets'] = data['tweet'].apply(lambda x: prepare_text(x))

In [ ]:
data.head(5)

## Tfidf for features

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
corpus = data['clean_tweets'].values.astype('U')

In [17]:
stopwords = set(nltk_stopwords.words('english'))

In [18]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

In [19]:
import pickle

In [21]:
pickle.dump(count_tf_idf, open("tf_idf.pkt", "wb"))

In [22]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(
    tf_idf, data['Toxicity'], test_size = 0.8, random_state= 42, shuffle=True
)

## Create a Binary Classification Model

In [23]:
model_bayes = MultinomialNB()

In [24]:
model_bayes = model_bayes.fit(tf_idf_train, target_train)

In [25]:
y_pred_proba = model_bayes.predict_proba(tf_idf_test)[::, 1]

In [ ]:
y_pred_proba

In [27]:
fpr, tpr, _ = roc_curve(target_test, y_pred_proba)

In [28]:
final_roc_auc = roc_auc_score(target_test, y_pred_proba)

In [ ]:
final_roc_auc

In [ ]:
test_text = "I hate you moron"
test_tfidf = count_tf_idf.transform([test_text])
display(model_bayes.predict_proba(test_tfidf))
display(model_bayes.predict(test_tfidf))

## Save the model

In [32]:
pickle.dump(model_bayes, open("toxicity_model.pkt", "wb"))